In [2]:
using DataFrames
using Chemfiles

home = "/home/german/labo/18/egfr/model/"
rehome = "/home/german/labo/18/egfr/modos/"

"/home/german/labo/18/egfr/model/"

In [20]:
function res_3_to_1(in_3::AbstractString)
    
    res = lowercase(in_3)
    out_1 = AbstractString
    
    if res == "ala"
        out_1 = "A"
    elseif res == "arg"
        out_1 = "R"
    elseif res == "asn"
        out_1 = "N"
    elseif res == "asp"
        out_1 = "D"
    elseif res == "cys"
        out_1 = "C"
    elseif res == "gln"
        out_1 = "Q"
    elseif res == "glu"
        out_1 = "E"
    elseif res == "gly"
        out_1 = "G"
    elseif res == "his"
        out_1 = "H"
    elseif res == "ile"
        out_1 = "I"
    elseif res == "leu"
        out_1 = "L"
    elseif res == "lys"
        out_1 = "K"
    elseif res == "met"
        out_1 = "M"
    elseif res == "phe"
        out_1 = "F"
    elseif res == "pro"
        out_1 = "P"
    elseif res == "ser"
        out_1 = "S"
    elseif res == "thr"
        out_1 = "T"
    elseif res == "trp"
        out_1 = "W"
    elseif res == "tyr"
        out_1 = "Y"
    elseif res == "val"
        out_1 = "V"
    else
        error("Unknown aminoacid: ", in_3)
    end
    
    return out_1
end

function res_3_to_1(in_vec::Array{String, 1})

    out_vec = Array{String, 1}(length(in_vec))
    i = 0
    for each in in_vec
        i+=1
        try 
            out_vec[i] = res_3_to_1(each)
        catch e
            error("Malformed array list.\nCaught error: $e")
        end
    end
    
    return out_vec
end

res_3_to_1 (generic function with 2 methods)

In [21]:
function seq_from_top(in_top::Chemfiles.Topology)
    # Get 1 letter code amino acid list from a topology
    aa = convert(Int64, count_residues(in_top))
    ids = Array{Int64, 1}(aa)

    for i=1:aa
        ids[i] = id(Residue(in_top, i-1))
    end
    ids_sorted_indices = sortperm(ids);

    seq_3 = Array{String, 1}(aa)
    i = 0
    for each in ids_sorted_indices
        i+=1
        seq_3[i] = name((Residue(in_top, each-1)))
    end
    seq_1 = res_3_to_1(seq_3);
    
    return seq_1
end

seq_from_top (generic function with 1 method)

In [4]:
tmp = readdlm(string(home, "/pdbs.list"))
pdbs_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

# Leo alineamiento

In [9]:
alineamientos = Array{Array{Any, 2}, 1}(length(pdbs_list))
indices_mut = Array{Array{Int64, 1}, 1}(length(pdbs_list))
indices_mis = Array{Array{Int64, 1}, 1}(length(pdbs_list))

j = 0

for each in pdbs_list
    j += 1
    println(each) 
    
    in_seq_canonical = Array{AbstractString, 1}
    in_seq_each = Array{AbstractString, 1}
    in_mut_each = Array{AbstractString, 1}

    seq_canonical = Array{AbstractString, 1}
    seq_each = Array{AbstractString, 1}
    mut_each = Array{AbstractString, 1}
    
    
    # Leo archivo.
    in_filename = string(home, each, "/alignment_", each)
    alignment_file = open(in_filename)
    lineas = readlines(alignment_file)

    k = 0
    # Ahora leo todas las líneas y tomo los arrays en bruto
    for linea in lineas
        k+=1
        inicio = 31
        final = length(lineas) - 3
    
        if inicio < k < final
            n_linea = k - 31
            altura = ((n_linea - 1 + 4) % 4)
        
            if altura == 0
                in_seq_canonical = [ in_seq_canonical ; linea[22:end-7] ]
            elseif altura == 1
                in_mut_each = [ in_mut_each ; linea[22:end] ]
            elseif altura == 2
                in_seq_each = [ in_seq_each ; linea[22:end-7] ]
            end
        end
    end
    
    # Paso las secuencias en bruto a arrays bien formados.
    for i = 2:length(in_seq_canonical)
        tmp_canonical = [ in_seq_canonical[i][j] for j = 1:length(in_seq_canonical[i]) ]
        seq_canonical = [ seq_canonical ; tmp_canonical]
    
        tmp_each = [ in_seq_each[i][j] for j = 1:length(in_seq_each[i]) ]
        seq_each = [ seq_each ; tmp_each]

        tmp = [ in_mut_each[i][j] for j = 1:length(in_mut_each[i]) ]
        mut_each = [ mut_each ; tmp]
    end

    # Determino q pdbs tienen mutaciones y en q posiciones.
    tmp_indices_mut = findin(mut_each, ".")
    indices_mut[j] = tmp_indices_mut
    
    # Determino las regiones a modelar
    tmp_indices_mis = findin(mut_each, " ") .- 1
    indices_mis[j] = tmp_indices_mis

    # Guardo todo
    alineamiento = [ seq_canonical seq_each mut_each ]
    alineamientos[j] = alineamiento 
end

1M14_A
1M17_A
1XKK_A
2EB2_A
2GS2_A
2GS6_A
2GS7_A
2ITN_A
2ITP_A
2ITU_A
2ITX_A
2ITZ_A
2J5F_A
2RGP_A
3BEL_A
3GOP_A
3GT8_A
3GT8_C
3IKA_A
3IKA_B
3LZB_A
3UG1_A
3VJN_A
3W2O_A
3W2R_A
3W2S_A
3W32_A
3W33_A
4G5J_A
4I1Z_A
4I22_A
4I23_A
4I24_A
4LI5_A
4LQM_A
4R3P_A
4R5S_A
4RJ4_A
4ZAU_A
4ZJV_A
5C8K_A
5CAO_A
5CAP_A
5CAV_A
5CNN_A
5CNN_B
5CNO_A
5CZH_A
5EDP_A
5EM5_A
5EM6_A
5EM7_A
5EM8_A
5HG5_A
5HG7_A
5HG8_A
5HIB_A
5HIC_A


### Renumero los indices missings. La numeración original es según
### las posiciones del alineamiento. Quiero obtener la numeración según
### la sequencia canonical

In [12]:
mod_indices_mis = Array{Array{Int64, 1}, 1}(length(pdbs_list))

for i = 1:length(pdbs_list)
    tmp_indices = Array{Int64}[]
    
    # La secuencia a modelar empieza y termina con leucina
    pre_mask = findin(alineamientos[i][:, 1], "L")
    mask = pre_mask[1] .< indices_mis[i] .< pre_mask[end]
    mod_indices_mis[i] = indices_mis[i][mask] .- pre_mask[1] .+ 2
    # Ya tengo los índices de missings numerados según la seq canonical.
end

# Determino la posición en la q empieza y termina la secuencia de relevancia

In [235]:
tmp_seq = zeros(317, length(pdbs_list))

starts = Array{Int64}(length(pdbs_list))
ends = Array{Int64}(length(pdbs_list))
check_mask_start = ['L' ; 'L' ; 'R' ]
check_mask_end = ['R' ; 'Y' ; 'L' ]

for j = 1:length(pdbs_list)
    # Saco 1 pq los arrays empiezan con un elemento undef
    tmp_struct = alineamientos[j][2:end, 2]
    tmp_struct = tmp_struct[tmp_struct .!= '-']
    for i = 1:50
        if check_mask_start == tmp_struct[i:i+2]
            starts[j] = i
            break
        end
    end
    for i = 250:length(tmp_seq)
        if check_mask_end == tmp_struct[i:i+2]
            ends[j] = i + 2
            break
        end
    end
end

## Leo bfactors de PDBs originales, sin completar

In [288]:
# 3W32_A, con 317 aa's, es el modelo más largo
proto_bfactors = zeros(317, length(pdbs_list))
i = 3
for i = 1:length(pdbs_list)
    each = pdbs_list[i]
    in_filename = string(home, each, "/", each, ".pdb")
    in_file = open(in_filename)

    j = 0
    for linea in eachline(in_file)
        try
            if linea[14:15] == "CA"
                j += 1
                proto_bfactors[j, i] = parse(Float64, linea[61:66])
            end
        catch e
            break
        end
    end
    
end

# Adapto los bfactors p/ q correspondan con la secuencia de relevancia

In [289]:
bfactors = Array{Array{Float64, 1}}(length(pdbs_list))

for i = 1:length(pdbs_list)
    if i == 18 || i == 33 || i == 40 || i == 46
        continue
    end
    
    tmp_mask = proto_bfactors[:, i] .!= 0
    bfactors[i] = proto_bfactors[tmp_mask, i][starts[i]:ends[i]];
end

In [290]:
proto_teo_bfactors = Array{Array{Float64, 1}}(length(pdbs_list))
i = 1
each = pdbs_list[i]
in_filename = string(rehome, each, "/bfteo_", each)
proto_teo_bfactors[i] = readdlm(in_filename)[:, 2]

277-element Array{Float64,1}:
 338.992 
 181.109 
 178.402 
 175.985 
 117.681 
 168.477 
 182.878 
 271.617 
 217.928 
 146.641 
 238.899 
 224.764 
 373.393 
   ⋮     
  80.0681
 111.82  
 198.712 
 101.188 
  96.7498
 175.072 
 176.627 
 117.144 
 159.561 
 174.331 
 112.087 
  98.6368

In [291]:
bfactors[i][45:50]

6-element Array{Float64,1}:
 102.32
 101.79
 110.9 
 114.4 
 119.51
 116.63

In [292]:
cor(bfactors[i], proto_teo_bfactors[i])

0.6271720305353076